<a href="https://colab.research.google.com/github/marcorrea1/AAI2026/blob/main/prompt_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# ── SETUP ────────────────────────────────────────────────────
!pip install groq -q

from groq import Groq
from google.colab import userdata

client = Groq(api_key=userdata.get('GROQ_API_KEY'))

def call_ai(system_prompt, user_message):
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": user_message}
        ],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()

# ── MOVIE INFO ────────────────────────────────────────────────
movie = "Spider-Man No Way Home"
audience = "someone who has never seen the movie before"
max_attempts = 5

print("=" * 55)
print("  SELF-REFLECTION PROMPT — MOVIE SUMMARIZER")
print("=" * 55)
print(f"\n🎬 Movie: {movie}")
print(f"👤 Audience: {audience}")
print(f"🔁 Max revision attempts: {max_attempts}")

# ── STEP 1: GENERATE FIRST SUMMARY (BEFORE) ──────────────────
print("\n" + "=" * 55)
print("STEP 1 — GENERATING FIRST SUMMARY...")
print("=" * 55)

step1_output = call_ai("""
You are a movie summarizer. Write a basic summary of the movie.
Keep it around 80 words. Don't worry too much about quality yet.
Just write a first draft.
""", f"Summarize this movie: {movie}")

print("\n📝 ORIGINAL FIRST DRAFT:")
print("-" * 55)
print(step1_output)
print(f"\n📊 Word count: {len(step1_output.split())} words")

# ── REVISION LOOP ─────────────────────────────────────────────
current_summary = step1_output
attempt = 0
approved = False
all_versions = [step1_output]

while not approved and attempt < max_attempts:
    attempt += 1
    print("\n" + "=" * 55)
    print(f"REVISION CYCLE {attempt} of {max_attempts}")
    print("=" * 55)

    # ── CRITIQUE ─────────────────────────────────────────────
    print(f"\n🔍 CRITIQUE (Attempt {attempt}):")
    print("-" * 55)

    critique = call_ai(f"""
You are an extremely strict movie summary editor.
Critique this summary against ALL of these hard requirements:

1. ACCURACY — Does it correctly describe the main plot? Does it mention Peter Parker, Doctor Strange, the multiverse, and the villains? FAIL if any are missing.
2. CLARITY — Can someone with zero knowledge of Spider-Man understand it? FAIL if it assumes prior knowledge.
3. AUDIENCE — Is it written for a general audience aged 13 and up? FAIL if it uses complex jargon.
4. LENGTH — Is it STRICTLY between 80 and 95 words? Count carefully. FAIL if even 1 word over or under.
5. SPOILERS — Does it reveal the ending or major character deaths? FAIL if it spoils anything.
6. TONE — Is it engaging and interesting, not just a dry plot recap? FAIL if it sounds boring.
7. OPENING — Does it start with something other than the movie title or character name? FAIL if it starts with Spider-Man or Peter Parker.
8. SENTENCES — Are all sentences under 25 words each? FAIL if any sentence is too long.

For each criteria write: [PASS] or [FAIL] — one sentence reason.
Count the exact word count and show it.
End with VERDICT: APPROVED only if ALL 8 criteria pass. Otherwise VERDICT: REJECTED.
Be very strict — do not approve unless everything is perfect.
""", f"Summary to critique:\n{current_summary}")

    print(critique)

    # ── CHECK IF APPROVED ─────────────────────────────────────
    if "VERDICT: APPROVED" in critique.upper():
        approved = True
        print(f"\n✅ APPROVED on attempt {attempt}!")
        break

    # ── REFLECT ───────────────────────────────────────────────
    print(f"\n🤔 REFLECTION (Attempt {attempt}):")
    print("-" * 55)

    reflection = call_ai("""
You are a reflective writer. Based on the critique, identify exactly what needs to change.

Respond in this format:
FAILED CRITERIA: <list only the ones that failed>
ROOT CAUSE: <why did each one fail?>
FIX PLAN: <exactly what will be different in the next version>
TARGET WORD COUNT: <specific number between 80-95>

Be very specific. Do not be vague.
""", f"Summary:\n{current_summary}\n\nCritique:\n{critique}")

    print(reflection)

    # ── IMPROVE ───────────────────────────────────────────────
    print(f"\n✏️ IMPROVED VERSION (Attempt {attempt}):")
    print("-" * 55)

    current_summary = call_ai(f"""
You are a movie summarizer writing a revised version. You MUST fix every failed criteria.

Strict requirements — every single one must be met:
1. Between 80 and 95 words EXACTLY — count every word before submitting
2. Must mention: Peter Parker, Doctor Strange, the multiverse, and the villains
3. Written for someone who has never seen Spider-Man before
4. No spoilers — do not reveal the ending or any character deaths
5. Must NOT start with Spider-Man, Peter Parker, or the movie title
6. Every sentence must be under 25 words
7. Must sound engaging and interesting, not like a dry plot recap
8. Use simple language suitable for ages 13 and up

Write ONLY the revised summary. Nothing else. No labels, no explanations.
""", f"Movie: {movie}\n\nCurrent summary:\n{current_summary}\n\nCritique:\n{critique}\n\nFix plan:\n{reflection}")

    print(current_summary)
    print(f"\n📊 Word count: {len(current_summary.split())} words")
    all_versions.append(current_summary)

# ── FINAL RESULT ──────────────────────────────────────────────
print("\n" + "=" * 55)
print("         BEFORE vs AFTER COMPARISON")
print("=" * 55)

print("\n📝 ORIGINAL FIRST DRAFT:")
print("-" * 55)
for line in step1_output.split('. '):
    if line.strip():
        print(f"  {line.strip()}.")
print(f"\n  📊 Word count: {len(step1_output.split())} words")

print(f"\n✅ FINAL APPROVED VERSION (after {attempt} revision(s)):")
print("-" * 55)
for line in current_summary.split('. '):
    if line.strip():
        print(f"  {line.strip()}.")
print(f"\n  📊 Word count: {len(current_summary.split())} words")

# ── AUTO CHECKS ───────────────────────────────────────────────
print("\n" + "=" * 55)
print("            AUTO QUALITY CHECKS")
print("=" * 55)
word_count = len(current_summary.split())
print(f"  Word count (80-95)    : {word_count} words — {'✅ PASS' if 80 <= word_count <= 95 else '❌ FAIL'}")
print(f"  Mentions multiverse   : {'✅ PASS' if 'multiverse' in current_summary.lower() else '❌ FAIL'}")
print(f"  Mentions Doctor Strange: {'✅ PASS' if 'strange' in current_summary.lower() else '❌ FAIL'}")
print(f"  No title at start     : {'✅ PASS' if not current_summary.startswith('Spider-Man') else '❌ FAIL'}")
print(f"  No bullet points      : {'✅ PASS' if '-' not in current_summary else '❌ FAIL'}")
print(f"  Status                : {'✅ APPROVED' if approved else '⚠️ MAX ATTEMPTS REACHED'}")

# ── SUMMARY ───────────────────────────────────────────────────
print("\n" + "=" * 55)
print("       SELF REFLECTION CYCLE COMPLETE")
print("=" * 55)
print(f"  Total revision cycles : {attempt}")
print(f"  Versions generated    : {len(all_versions)}")
print(f"  Final status          : {'✅ APPROVED' if approved else '⚠️ Max attempts reached'}")
print("\n  Steps completed:")
print("  Step 1 — GENERATE  : Wrote first draft")
print("  Step 2 — CRITIQUE  : Checked against 8 strict criteria")
print("  Step 3 — REFLECT   : Identified exact issues")
print("  Step 4 — IMPROVE   : Rewrote based on feedback")
print("  Step 5 — LOOP      : Repeated until approved or max attempts")
print("\n✅ Self reflection cycle done!")
print("=" * 55)

  SELF-REFLECTION PROMPT — MOVIE SUMMARIZER

🎬 Movie: Spider-Man No Way Home
👤 Audience: someone who has never seen the movie before
🔁 Max revision attempts: 5

STEP 1 — GENERATING FIRST SUMMARY...

📝 ORIGINAL FIRST DRAFT:
-------------------------------------------------------
Spider-Man: No Way Home follows Spider-Man as he deals with the aftermath of his identity being revealed. With help from Doctor Strange, he tries to reverse the damage, but it opens up the multiverse, bringing in villains from other universes, forcing Spider-Man to confront them and find a way to restore balance.

📊 Word count: 52 words

REVISION CYCLE 1 of 5

🔍 CRITIQUE (Attempt 1):
-------------------------------------------------------
1. ACCURACY — [FAIL] because it doesn't mention Peter Parker explicitly.
2. CLARITY — [FAIL] because it assumes prior knowledge of Spider-Man's identity being revealed.
3. AUDIENCE — [PASS] because it uses simple language.
4. LENGTH — [FAIL] because it has 96 words, exceeding t

# Exercise 3 — Self Reflection Prompt for Improving Output

## Tool Used
Google Colab + Groq API (LLaMA 3.3 model)

## What This Does
I asked the AI to write a movie summary for Spider-Man No Way Home, then had it
critique its own work and keep fixing it until it met all the requirements.

---

## My Prompts By Step

**Step 1 — Generate First Draft**
"Hey can you write me a short summary of Spider-Man No Way Home,
keep it around 80 words, just a basic first draft is fine"

**Step 2 — Critique**
"Ok so can you check this summary and tell me if its actually good?
I need you to be really strict about it. Check if it gets the plot right,
if someone who never saw the movie would understand it, if its the right
length between 80 and 95 words, if it gives away any spoilers, if it sounds
interesting and not boring, and if the sentences are too long.
Tell me pass or fail for each one and dont go easy on it"

**Step 3 — Reflect**
"Based on that critique can you tell me exactly what went wrong and
what needs to be fixed in the next version? I need to know the specific
problems not just general feedback"

**Step 4 — Improve**
"Ok now rewrite it and fix everything that failed. It has to be between
80 and 95 words, mention Peter Parker, Doctor Strange, the multiverse and
the villains, no spoilers, cant start with Spider-Man or Peter Parker,
keep sentences short, and make it sound actually interesting.
Just give me the summary nothing else"

**Step 5 — Loop Until Approved**
"Keep doing steps 2 through 4 until every single criteria passes.
Don't stop until it gets a full approved verdict or hits 5 attempts"

---

## How Each Step Uses Prior Output
- Step 2 critiques the summary from Step 1
- Step 3 reflects on the failed criteria from Step 2
- Step 4 rewrites using the original summary plus the critique and reflection from Steps 2 and 3
- Step 5 loops back to Step 2 each time using the latest improved version until everything passes

---

## Iteration Notes
The first version of this only had 5 criteria and did not loop, it just revised once.
After testing I noticed the AI was passing itself too easily so I made the criteria
stricter by adding 3 more checks for spoilers, tone, and sentence length.
I also added the revision loop so it keeps going until it actually meets every
requirement instead of just doing one revision and stopping.